## Recherche d'une bonne fonction de coût pour le modèle DELTA_DIST+H_05

On a remarqué que les modèles prédisent systématiquement des valeurs similaires qui minimisent en moyenne la fonction de coût mais qui ne sont pas liées à la géométrie des molécules.

Pour parer à ce problème, nous allons définir ici des fonctions de coût alternatives qui auront pour but de pénaliser les modèles s'ils s'approchent de la "solution de facilité" qu'est la prédiction systématique d'une valeur minimisant en moyenne la fonction de coût.

Pour évaluer les différentes fonctions de coût, nous utiliserons toujours le RMSE partiel actuel comme mesure de validation. Si nous trouvons une fonction qui entraîne un modèle dont les prédictions ont un RMSE partiel inférieur à 107, alors cette fonction de coût sera considérée comme intéressante.


#### Chemin des fichiers

In [12]:
train_prepared_input_loc = "../data/train_set_riken_v2_prepared_input_bruit+_anums_reduced.h5"
train_labels_loc = "../data/train_set_riken_v2_labels_bruit+_anums_reduced.h5"

mini_prepared_input_loc = "../data/mini_set_prepared_input_bruit+_anums_reduced.h5"
mini_labels_loc = "../data/mini_set_labels_bruit+_anums_reduced.h5"

models_loc = "../models/DELTA_DIST+H_05/8.5/"
logs_loc = "../models/DELTA_DIST+H_05/8.5/"

### Définition des fonctions de coût

#### Calcul du masque

In [1]:
import tensorflow as tf

def calcul_masque_atomes_definis(targets):
    """ On calcule le masque booléen des atomes donnés en entrée du RN en fonction du vecteur targets"""
    
    # On cherche à obtenir un masque booléen des atomes définis en entrée. Pour cela, on prend en entrée
    # les étiquettes sous la forme d'une matrice (200, 4) dont chaque ligne i est la distance de l'atome i avec
    # les atomes fictifs du repère. L'atome est indéfini ssi. la somme de la ligne est nulle. En effet,
    # un atome défini ne peut pas avoir une distance nulle avec les quatre atomes fictifs, et on veille
    # à ce que le vecteurs targets ne contienne que des valeurs nulles pour les atomes non définis.
    # On obtient donc un masque booléen de tous les atomes définis en entrée
    
    ## On somme les distances de chaque atome ##
    targets_dists_sums = tf.reduce_sum(targets, 1)
    
    ## On créé le masque des sommes différentes de zéro ##
    
    # Création des matrice de True et de False de la dimension de la matrice des sommes (nécessaires
    # pour tf.where)
    zeros = tf.cast(tf.zeros_like(targets_dists_sums),dtype=tf.bool)
    ones = tf.cast(tf.ones_like(targets_dists_sums),dtype=tf.bool)
    
    return tf.where(targets_dists_sums>0, ones, zeros)


/home/etudiant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Calcul du RMSE partiel en tant que fonction de validation

In [14]:
def partial_rmse(predictions, targets, inputs):
    """ Calcule le RMSE partiel des prédictions par rapport aux valeurs attendues. Le RMSE est partiel car
    on ne le calcule que pour les sorties correspondant aux atomes donnés en entrée. En d'autres
    termes, on ne pousse pas le modèle à donner des distances nulles pour les atomes indéfinis
    en entrée"""
    
    with tf.name_scope("partial_rmse_validation"):

        # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
        predictions = tf.reshape(predictions, [-1, 4])
        targets = tf.reshape(targets, [-1, 4])

        # On calcule le masque des atomes définis selon les cibles
        defined_atoms_mask = calcul_masque_atomes_definis(targets)
        
        # On masque les prédictions et les étiquettes selon le masque des atomes définis
        targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
        predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)   

        return tf.sqrt(tf.reduce_mean(tf.squared_difference(predictions_masked, targets_masked)), name="rmse")
    

#### Pénalisation des modèles effectuant des prédictions avec un écart type faible

Afin d'éviter que le modèle prédise systématiquement les mêmes valeurs, nous allons pénaliser les modèles prédisant des valeurs trop proches les unes des autres comme c'est le cas actuellement. La fonction de coût que nous définissons ici est définie de la façon suivante : coût(pred) = RMSE(pred)\*(1+(1/σ)), σ étant l'écart type de la prédiction.

In [55]:
def partial_rmse_std_dev_penalty(predictions, targets):
    
    # On calcule le masque des atomes définis selon les cibles
    defined_atoms_mask = calcul_masque_atomes_definis(targets)
    predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)   
    
    # On calcule la moyenne des prédictions
    mean = tf.reduce_mean(predictions_masked)
    
    # On calcule l'écart type
    stddev = tf.sqrt(tf.reduce_mean(tf.squared_difference(predictions_masked, mean)))
    
    tf.al
    
    stddev = tf.Print(stddev, [stddev])
    
    # On calcule le coefficient que l'on ajoute au RMSE selon notre formule
    coef = tf.ones_like(stddev) + tf.divide(tf.ones_like(stddev), stddev)
    
    coef = tf.Print(coef, [coef])
    
    # On calcul le coût selon notre formule
    return tf.multiply(partial_rmse(predictions, targets, None), coef)

## Préparation des données


#### Fonction renvoyant deux sous-ensembles du jeu d'entrainement : un ensemble d'exemples et les cibles correspondantes


In [56]:
def get_fold(train_set, targets, reduce_train_fold_size):
    """ Permet d'obtenir un sous-ensemble du jeu d'entraînement afin de ne pas travailler sur le jeu
    d'entraînement total pour la recherche par quadrillage et donc de gagner du temps d'exécution. L'idée
    et que si un ensemble d'hyper-paramètres produit des meilleurs résultats que les autres ensembles
    d'hyper-paramètres sur l'ensemble du jeu d'entraînement, alors on suppose que ce sera également 
    le cas sur une partie des données. """

    return (train_set["inputs"][:reduce_train_fold_size], targets["targets"][:reduce_train_fold_size])
    

## Entraînement des modèles

On va ici entraîner des modèles utilisant les fonctions de coût définies pour évaluer leurs performances, toujours en prenant le RMSE partiel utilisé dans les modèles précédents comme référence

### Création du RN

In [57]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.optimizers import Adam
from tflearn.data_preprocessing import DataPreprocessing
import math


def creer_RN(epsilon=1e-8, learning_rate=0.001, dropout_val=0.99, stddev_init=0.001,
             hidden_act='relu', outlayer_act='prelu', weight_decay=0.001, width=360, depth=3,
             validation_fun=partial_rmse, cost_fun=partial_rmse_std_dev_penalty):
    """ Fonction créant un réseau de neurones de type fully connected, ayant une couche d'entrée de 360
    neurones, quatre couches cachées de 360 neurones et une sortie de 240 neurones
    Inputs : hyperparamètres
    """

    # On créé l'initialisateur de tenseur avec une loi normale tronquée. sigma = stddev_init, et les 
    # valeurs à plus de 2sigma sont re-tirées
    winit = tfl.initializations.truncated_normal(stddev=stddev_init, dtype=tf.float32, seed=None)
    
    # On créé l'input du RN
    network = input_data(shape=[None, 360], name='input')
    
    # On créé les couches cachées
    for i in range(depth):
        network = fully_connected(network, width, activation=hidden_act, name='fc'+str(i), weights_init=winit,
                                  weight_decay=weight_decay)
        # On détruit des neurones aléatoirement avec une la probabilité donnée en entrée
        network = dropout(network, dropout_val)
    
    # On ajoute la couche de sortie du réseau
    # Fonction d'activation prelu
    # Initilisée avec la loi normale tronquée
    network = fully_connected(network, 240, activation=outlayer_act, name='outlayer', weights_init=winit)
    
    adam = Adam(learning_rate=learning_rate, epsilon=epsilon)
    
    # Couche d'évaluation du modèle. Utilisation d'une descente stochastique Adam
    # Learning rate = 0.05
    # Loss = fonction définie rmse
    network = regression(network, optimizer=adam,
    loss=cost_fun, metric=validation_fun, name='target')
            
    return network

#### Définition d'une fonction d'entraînement d'un modèle

In [58]:
import h5py
import tflearn as tfl
import time
from scipy import sparse
import numpy as np
import gc
import tensorflow as tf


def train_model(input_X, labels_y, model_name, model_path, logs_path, samples_per_batch=1000, epochs=5,
                learning_rate=0.001, epsilon=1e-8, dropout=0.99, stddev_init=0.001, hidden_act='relu',
                outlayer_act='prelu', cost_fun=partial_rmse_std_dev_penalty, validation_fun=partial_rmse):
    
    total_start_time = time.time()

    tf.reset_default_graph()
    
    # On créé le réseau 
    network = creer_RN(learning_rate=learning_rate, epsilon=epsilon, dropout_val=dropout,
                       stddev_init=stddev_init, hidden_act=hidden_act, outlayer_act=outlayer_act, width=1000,
                       validation_fun=validation_fun, cost_fun=cost_fun)

    # On créé le modèle
    model = tfl.DNN(network, tensorboard_verbose=3, tensorboard_dir=logs_path)

    # Entraînement
    model.fit(X_inputs=input_X,Y_targets=labels_y, batch_size=samples_per_batch,
              shuffle = True, snapshot_step=100, validation_set=0.1,
              show_metric=True, run_id=model_name, n_epoch=epochs)

    # Sauvegarde du modèle
    #model.save(model_path + model_name + ".tflearn")


#### Entraînement du modèle pénalisant les prédictions avec un faible écart-type

In [59]:
import h5py


input_X_h5 = h5py.File(train_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(train_labels_loc, 'r')

"""
input_X_h5 = h5py.File(mini_prepared_input_loc, 'r')
labels_y_h5 = h5py.File(mini_labels_loc, 'r')
"""
input_X, labels_y = get_fold(input_X_h5, labels_y_h5, 5000000)

model_name = "DELTA_DIST+H_05_cost_stddev_penalty"
model_path = models_loc
logs_path = logs_loc

train_model(input_X, labels_y, model_name, model_path, logs_path, samples_per_batch=1000, 
            epochs=100, learning_rate=0.01, dropout=0.97, epsilon=0.001, hidden_act="elu",
            outlayer_act="linear", validation_fun=partial_rmse, cost_fun=partial_rmse_std_dev_penalty)


/home/etudiant/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


---------------------------------
Run id: DELTA_DIST+H_05_cost_stddev_penalty
Log directory: ../models/DELTA_DIST+H_05/8.5/
INFO:tensorflow:Summary name partial_rmse/ (raw) is illegal; using partial_rmse/__raw_ instead.
---------------------------------
Training samples: 1
Validation samples: 1
--
Training Step: 1  | time: 1.564s
| Adam | epoch: 001 | loss: 0.00000 - partial_rmse/rmse: 0.0000 | val_loss: 227.81790 - val_acc: 196.7698 -- iter: 1/1
--
Training Step: 2  | total loss: 438156.31250 | time: 1.119s
| Adam | epoch: 002 | loss: 438156.31250 - partial_rmse/rmse: 141.9335 | val_loss: 714.72870 - val_acc: 713.7233 -- iter: 1/1
--
Training Step: 3  | total loss: 79833.18750 | time: 1.104s
| Adam | epoch: 003 | loss: 79833.18750 - partial_rmse/rmse: 155.6030 | val_loss: 1558.45740 - val_acc: 1557.6603 -- iter: 1/1
--
Training Step: 4  | total loss: 20225.46875 | time: 1.102s
| Adam | epoch: 004 | loss: 20225.46875 - partial_rmse/rmse: 305.3456 | val_loss: 2126.01099 - val_acc: 2125.

Training Step: 41  | total loss: 54.71764 | time: 1.082s
| Adam | epoch: 041 | loss: 54.71764 - partial_rmse/rmse: 54.1972 | val_loss: 386.62140 - val_acc: 385.6464 -- iter: 1/1
--
Training Step: 42  | total loss: 64.41997 | time: 1.127s
| Adam | epoch: 042 | loss: 64.41997 - partial_rmse/rmse: 63.9029 | val_loss: 306.71939 - val_acc: 305.6500 -- iter: 1/1
--
Training Step: 43  | total loss: 69.29197 | time: 1.102s
| Adam | epoch: 043 | loss: 69.29197 - partial_rmse/rmse: 68.7869 | val_loss: 232.82098 - val_acc: 231.3691 -- iter: 1/1
--
Training Step: 44  | total loss: 62.49785 | time: 1.082s
| Adam | epoch: 044 | loss: 62.49785 - partial_rmse/rmse: 62.0460 | val_loss: 216.76106 - val_acc: 214.9614 -- iter: 1/1
--
Training Step: 45  | total loss: 61.21859 | time: 1.116s
| Adam | epoch: 045 | loss: 61.21859 - partial_rmse/rmse: 60.7358 | val_loss: 218.88394 - val_acc: 217.1404 -- iter: 1/1
--
Training Step: 46  | total loss: 64.85848 | time: 1.085s
| Adam | epoch: 046 | loss: 64.85848 -

KeyboardInterrupt: 